<a href="https://colab.research.google.com/github/myazzeh/NLP-Course/blob/main/Sequence_Learning/NLP_seq2seq_machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import collections
import helper
import numpy as np
import os
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential
from keras.optimizers import Adam
import pandas as pd
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Embedding
from keras.losses import sparse_categorical_crossentropy

#**Download and Prepare the datasets**#

In [ ]:
!wget https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/small_vocab_en
!wget https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/small_vocab_fr

--2024-05-07 08:13:56--  https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/small_vocab_en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9085267 (8.7M) [text/plain]
Saving to: ‘small_vocab_en’

small_vocab_en      100%[===================>]   8.66M  --.-KB/s    in 0.1s    

2024-05-07 08:13:57 (61.3 MB/s) - ‘small_vocab_en’ saved [9085267/9085267]

--2024-05-07 08:13:57--  https://raw.githubusercontent.com/myazzeh/NLP-Course/main/datasets/small_vocab_fr
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10135742 (9.

In [ ]:
eng_file = os.path.join('small_vocab_en')
eng = open(eng_file, "r")
english_sentences= eng.read().split('\n')

fre_file =os.path.join('small_vocab_fr')
fre = open(fre_file, "r")
french_sentences = fre.read().split('\n')


In [ ]:
#Test the sentences
for i in range(2):
    print(f'English sentence at Line {i + 1}: {english_sentences[i]}')
    print(f'French sentence at Line {i + 1}: {french_sentences[i]}')
    print('-------------------------------------------------')

English sentence at Line 1: new jersey is sometimes quiet during autumn , and it is snowy in april .
French sentence at Line 1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
-------------------------------------------------
English sentence at Line 2: the united states is usually chilly during july , and it is usually freezing in november .
French sentence at Line 2: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
-------------------------------------------------


In [ ]:
# Get Initial Vocab Size for both English and French dataset
#english_vocab = len(collections.Counter([word for sentence in english_sentences for word in sentence.split()]))
#french_vocab  = len(collections.Counter([word for sentence in french_sentences for word in sentence.split()]))
#print(f'English vocab size is {english_vocab}, and French vocab size is {french_vocab}')

#**Tokenize English and French Corpus and Align all sequences**#

In [ ]:
# Find Maximum sequence Length in both datasets
length_eng = max([len(sentence) for sentence in english_sentences])
length_fre = max([len(sentence) for sentence in french_sentences])
max_length= max(length_eng, length_fre)
print(f'Maxmium sequence length in both dataset is {max_length}')

Maxmium sequence length in both dataset is 114


In [ ]:
etok = Tokenizer(char_level = False, oov_token='[UNK]')
etok.fit_on_texts(english_sentences)
english_seq = etok.texts_to_sequences(english_sentences)
english_seq = pad_sequences(english_seq, maxlen = max_length, padding = 'post')

ftok = Tokenizer(char_level = False)
ftok.fit_on_texts(french_sentences)
french_seq = ftok.texts_to_sequences(french_sentences)
french_seq = pad_sequences(french_seq, maxlen = max_length, padding = 'post')
#french_seq = french_seq.reshape(*french_seq.shape, 1)

In [ ]:
english_seq[0]

array([18, 24,  2,  9, 68,  5, 40,  8,  4,  2, 56,  3, 45,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [ ]:
french_seq[0]

array([ 35,  34,   1,   8,  67,  37,  11,  24,   6,   3,   1, 112,   2,
        50,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
print(*english_seq.shape)

137861 114


In [ ]:
#Get Vocab Size
english_vocab = len(etok.word_index)
french_vocab = len(ftok.word_index)
print(f'English vocab size is {english_vocab}, and French vocab size is {french_vocab}')

English vocab size is 200, and French vocab size is 344


#**First Machine Translation Model using Two RNN (Encoder and Decoder)**#

In [ ]:
#Model 1 is a simple RNN
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=english_vocab,output_dim=128,input_length=max_length))
model.add(Bidirectional(LSTM(256,return_sequences=False)))
model.add(RepeatVector(max_length))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(TimeDistributed(Dense(french_vocab,activation='softmax')))
learning_rate = 0.005
model.compile(loss = sparse_categorical_crossentropy, optimizer = Adam(learning_rate), metrics = ['accuracy'])
model.fit(english_seq, french_seq, batch_size = 1024, epochs = 2, validation_split = 0.2)

Epoch 1/2
  5/108 [>.............................] - ETA: 1:11:39 - loss: 2.7402 - accuracy: 0.7124

InvalidArgumentError: Graph execution error:

Detected at node sequential_1/embedding_1/embedding_lookup defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/u

In [ ]:
print(logits_to_text(model.predict(french_seq[:1])[0], ftok))

In [ ]:
y_id_to_word = {value: key for key, value in ftok.word_index.items()}
y_id_to_word[0] = '<PAD>'
sentence = 'he saw a old yellow truck'
sentence = [etok.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=max_length, padding='post')
sentences = np.array([sentence[0], english_seq[0]])
predictions = model.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in french_seq[0]]))

#**Second Machine Translation Model using Two RNN (Encoder and Decoder)**#





In [ ]:
#Model 2: Embedding

from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1])
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model
#tests.test_embed_model(embed_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
embeded_model = embed_model(
tmp_x.shape,
max_french_sequence_length,
english_vocab_size,
french_vocab_size)
embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

#**Third Machine Translation Model using Two RNN (Encoder and Decoder)**#

In [ ]:
#Model 3: Bidirectional RNNs
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1),
    input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy,
    optimizer = Adam(learning_rate),
    metrics = ['accuracy'])
    return model
tests.test_bd_model(bd_model)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
tmp_x.shape,
preproc_french_sentences.shape[1],
len(english_tokenizer.word_index)+1,
len(french_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

#**Forth Machine Translation Model using Two RNN (Encoder and Decoder)**#

In [ ]:
#Model 4: Encoder-Decoder

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(LSTM(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(LSTM(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, optimizer = Adam(learning_rate), metrics = ['accuracy'])
    return model

tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
tmp_x.shape,
preproc_french_sentences.shape[1],
len(english_tokenizer.word_index)+1,
len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

#**Fifth Machine Translation Model using Two RNN (Encoder and Decoder)**#

In [ ]:
#Model 5: Custom
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    model.compile(loss = sparse_categorical_crossentropy,
    optimizer = Adam(learning_rate),
    metrics = ['accuracy'])
    return model
tests.test_model_final(model_final)
print('Final Model Loaded')
def final_predictions(x, y, x_tk, y_tk):
tmp_X = pad(preproc_english_sentences)
model = model_final(tmp_X.shape,
preproc_french_sentences.shape[1],
len(english_tokenizer.word_index)+1,
len(french_tokenizer.word_index)+1)

model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = '<PAD>'
sentence = 'he saw a old yellow truck'
sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
sentences = np.array([sentence[0], x[0]])
predictions = model.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)